# Do It Yourself: Python With Partially Charged Batteries

<span class="hl">
Abhabongse Janthong **· Plane** <br/>
Watcharapol Watcharawisetkul **· Group**
</span>

<small>Kasikorn Business Technology Group</small>

In [ ]:
autoplay = False  # YouTube autoplay

<h1 class="center">“Batteries Included”</h1>

For the next few slides, we explain the slogan “Batteries Included” in Python

1. A video on toys without batteries included (put slogan into context).
2. This talk will turn **“partially charged”** batteries into **“fully-charged”**.

In [ ]:
from talk_resources import YouTubeVideo
display(YouTubeVideo('foWwW1_CFw4', autoplay=autoplay))

<h1 class="center">Charging Batteries</h1>

&nbsp;

<div class="center smcp text-120">
from &nbsp; <i class="fa-battery-1 fa-lg fa" style="color: #D66;"></i> &nbsp; 
to &nbsp; <i class="fa-battery-4 fa-lg fa" style="color: #6D6;"></i>
</div>

# 1. Built-in `range` function

Example

In [1]:
print(list(range(10)))
print(list(range(2, 10, 3)))
print(list(range(5, 0, -2)))
print(list(range(4, 1)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 5, 8]
[5, 3, 1]
[]


## Our implementation of `range`

In [2]:
def my_range(start, stop=None, step=1):
    """Implements built-in ``range()`` function."""
    
    if not isinstance(step, int):
        raise TypeError('step must be integer')
    if step == 0:
        raise ValueError('step cannot be zero')
    
    if stop is None:  # built-in syntax special case
        stop = start; start = 0    
    curr = start
    
    while (curr < stop) if (step > 0) else (curr > stop):
        yield curr
        curr += step
        
print(list(my_range(10)))
print(list(my_range(2, 10, 3)))
print(list(my_range(5, 0, -2)))
print(list(my_range(4, 1)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 5, 8]
[5, 3, 1]
[]


# 2. Filter-Map-Reduce

**Higher-order functions on sequences** <br/>
<small class="hl">_Sidenote:_ Using **comprehensions** could be a better option.</small>

In [3]:
from functools import reduce

values = [17, 4, 9, 13, 6, 11]

# filter: List of values greater than 10
print(list(filter(lambda x: x > 10, values)))

# map: Double each value in the list
print(list(map(lambda x: x * 2, values)))
print(list(map(lambda x, y: x + y, 'apx', 'bqy')))

# reduce: Find the product of all values
print(reduce(lambda x, y: x * y, values))

[17, 13, 11]
[34, 8, 18, 26, 12, 22]
['ab', 'pq', 'xy']
525096


## 2A. Our implementation of `filter`

In [4]:
def my_filter(cond_function, iterable):
    """Implements built-in ``filter()`` function."""
    
    if cond_function is None:
        cond_function = lambda value: value
    
    for value in iterable:
        if cond_function(value):
            yield value
            
print(list(my_filter(lambda x: x > 10, [17, 4, 9, 13, 6, 11])))
print(list(my_filter(None, [0, 1, True, False, None])))

[17, 13, 11]
[1, True]


## 2B. Our implementation of `map` <small>(simplified)<small>

1. Show `my_simple_map()` which works just like `map()` but accepting just one iterable.
    - A solution mimicks one from `my_filter()`
    - Mention another solution about how **generator expressions** can be used.
2. Show the generic `my_map()` implementation. This uses `zip` which may be considered cheating.
3. As an aside, show the implementation of `my_zip()`.

In [5]:
def my_simple_map(trans_function, iterable):
    """Implements built-in ``map()`` function accepting only one iterable."""

    for value in iterable:
        yield trans_function(value)
        
print(list(my_simple_map(lambda x: x * 2, [17, 4, 9, 13, 6, 11])))

[34, 8, 18, 26, 12, 22]


## 2C. Our implementation of full-fledged `map`

### Primer on variable \[un\]packing: `*`

In [6]:
def my_func(fst, snd, *rest):
    print(f'First = {fst}, Second = {snd}, Rest = {rest}')

my_func(1, 2, 3, 4, 5)
my_func(*[191, 199])
my_func(100, *range(2), 200)

First = 1, Second = 2, Rest = (3, 4, 5)
First = 191, Second = 199, Rest = ()
First = 100, Second = 0, Rest = (1, 200)


In [7]:
def my_map(trans_function, *iterables):
    """Implements built-in ``map()`` function."""
    
    if not iterables:
        raise TypeError("my_map() must have at least two arguments")
        
    for values_tuple in zip(*iterables):
        yield trans_function(*values_tuple)
        
print(list(my_map(lambda a, b: a * b, [10, 20, 30], [2, 3, 4])))
print(list(my_map(lambda x, y, z: x + y + z, range(4), range(4), range(4))))

[20, 60, 120]
[0, 3, 6, 9]


## Aside: our implementation of `zip`

In [8]:
def my_zip(*iterables):
    try:
        iterators = tuple(my_simple_map(iter, iterables))
        while True:
            values_list = []
            for it in iterators:
                values_list.append(next(it))
            yield tuple(values_list)
    except StopIteration:
        pass
    
print(list(my_zip([1, 2, 3, 4, 5], range(0, 10, 4))))

[(1, 0), (2, 4), (3, 8)]


## Yet another attempt to compact the code

In [9]:
def my_zip_other_attempt(*iterables):
    try:
        iterators = tuple(my_simple_map(iter, iterables))
        while True:
            # Warning: loops forever
            yield tuple(my_simple_map(next, iterators))
    except StopIteration:
        pass
    
print(list(my_zip_other_attempt([1, 2, 3, 4, 5], range(0, 10, 4))))

/home/kite/.local/share/virtualenvs/pycon-th-talks-3jG7hNy1/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'my_simple_map' raised StopIteration
  


KeyboardInterrupt: 

## 2D. Our implementation of `functools.reduce`

In [10]:
class _MISSING:
    pass

def my_reduce(accm_function, iterable, start=_MISSING):
    it = iter(iterable)
    accm = next(it) if start is _MISSING else start
    for value in it:
        accm = accm_function(accm, value)
    return accm

print(my_reduce(lambda a, b: a * b, range(1,10)))
print(my_reduce(lambda a, b: f"{a} {b}", ["Yes", "I", "can"], ">"))

362880
> Yes I can


# 3. Partially supplied arguments with `functools.partial`



In [18]:
from functools import partial
import sys

print_error = partial(print, "[error]", file=sys.stderr)

print_error("Hi!")
print_error("how", "are", "you" ,sep="_")

[error] Hi!
[error]_how_are_you


In [19]:
print(repr(partial))

<class 'functools.partial'>


### Our implementation of `functools.partial`

In [21]:
class my_partial(object):
    """Store function with partially pre-specified arguments."""
    
    def __init__(self, original_func, *args, **kwargs):
        self.original_func = original_func
        self.args = args
        self.kwargs = kwargs

    def __call__(self, *extra_args, **extra_kwargs):
        new_kwargs = self.kwargs.copy()
        new_kwargs.update(extra_kwargs)
        return self.original_func(*self.args, *extra_args, **new_kwargs)
    
    
print_error = my_partial(print, "[error]", file=sys.stderr)

print_error("Hi!")
print_error("how", "are", "you" ,sep="_")

[error] Hi!
[error]_how_are_you


# 4. Built-in `property` decorator function

Making getter/setter methods inside class definition
as if it is an instance property.

### Primer on decorators

In [11]:
from datetime import datetime
import sys

def log_usage(original_func):
    def new_func(*args, **kwargs):
        print(f"Called '{original_func.__name__}' at {datetime.now()}", 
              file=sys.stderr)
        original_func(*args, **kwargs)
    return new_func

@log_usage  # decorator modifies original function
def add_numbers(x, y):
    return x + y

print(add_numbers(10, 20))

None


Called 'add_numbers' at 2018-06-16 21:57:51.039656


### Example of `property` usage

In [12]:
class AspectRatioRectangle(object):
    """Rectaingle maintaining original ratio when resize."""
    
    def __init__(self, width, height):
        self.original_width = width
        self.original_height = height
        self.scale = 1
        
    @property
    def width(self):
        return self.original_width * self.scale
    
    @width.setter
    def width(self, new_width):
        self.scale = new_width / self.original_width
    
    @property
    def height(self):
        return self.original_height * self.scale
    
    @height.setter
    def height(self, new_height):
        self.scale = new_height / self.original_height
        
    def __repr__(self):
        return f"{type(self).__name__}({self.width}, {self.height})"

In [13]:
rect = AspectRatioRectangle(3, 4)
print(rect)
rect.height = 10
print(rect)
rect.width = 6
print(rect)

AspectRatioRectangle(3, 4)
AspectRatioRectangle(7.5, 10.0)
AspectRatioRectangle(6.0, 8.0)


In [14]:
class my_property(object):
    """
    Implements built-in ``property`` decorator function.
    Adapted from https://docs.python.org/3.6/howto/descriptor.html
    """
    def __init__(self, getter_fn):
        self.getter_fn = getter_fn
        self.setter_fn = None
        
    def __get__(self, instance, cls=None):
        if instance is None:
            return self          
        return self.getter_fn(instance)

    def __set__(self, instance, value):
        if self.setter_fn is None:
            raise AttributeError("cannot modify attribute")
        self.setter_fn(instance, value)

    def setter(self, setter_fn):
        self.setter_fn = setter_fn
        return self

In [15]:
class AspectRatioRectangle(object):
    
    def __init__(self, width, height):
        self.original_width = width
        self.original_height = height
        self.scale = 1
        
    @my_property
    def width(self):
        return self.original_width * self.scale
    
    @width.setter
    def width(self, new_width):
        self.scale = new_width / self.original_width
    
    @my_property
    def height(self):
        return self.original_height * self.scale
    
    @height.setter
    def height(self, new_height):
        self.scale = new_height / self.original_height
        
    def __repr__(self):
        return f"{type(self).__name__}({self.width}, {self.height})"

In [16]:
rect = AspectRatioRectangle(3, 4)
print(rect)
rect.height = 10
print(rect)
rect.width = 6
print(rect)

AspectRatioRectangle(3, 4)
AspectRatioRectangle(7.5, 10.0)
AspectRatioRectangle(6.0, 8.0)
